<a href="https://colab.research.google.com/github/earthtennison/SuperAILevel2/blob/main/AI_image_search_SiameseNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
import numpy as np
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import random
import urllib.request
import io
import random

# Load Data


In [ ]:
!gdown --id xxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [ ]:
!unzip "/content/trainKeys.zip"

In [ ]:
path = "/content/trainKeys"
for folder in os.listdir(path):
  print(folder,len( os.listdir(os.path.join(path,folder))  ))


In [ ]:
#df
!cd /content/&& gdown --id 1zb4vWj_YxMbKEEtBQ8zaNrVL5g9yC_e_ 

In [ ]:
!cd /content/&& unzip '/content/wazzadu.zip'

In [ ]:
cat = pd.read_csv("/content/dim_cat_subcat_tag_key.csv")
train_df = pd.read_csv("/content/train.csv")
test_df = pd.read_csv("/content/test-sample.csv")

###train test split

In [ ]:
from tqdm.auto import tqdm
from random import shuffle,sample

base_dir = "/content/trainKeys"
folder_list = [f for f in os.listdir(base_dir) if f[0]=='p']
print( len(folder_list), "categories found in the dataset")

# Train test split 
train_test_split = 0.8
no_of_files_in_each_class = 100

# Declare training array
cat_list = []
x = []
y = []
y_label = ''
index = 0

# Using just 100 images per category
for folder_name in tqdm(folder_list, desc = 'folder'):
    file_list = os.listdir(os.path.join(base_dir, folder_name))
    temp=[]

    #random
    file_list_pick = sample(file_list,no_of_files_in_each_class)
    for file_name in file_list_pick:
        temp.append(index)
        x.append(np.asarray(Image.open(os.path.join(base_dir, folder_name, file_name)).convert('RGB').resize((100, 100))))
        y.append(str(folder_name))
        index += 1
    cat_list.append(temp)

cat_list = np.asarray(cat_list)
x = np.asarray(x).astype(float)/255.0
y = np.asarray(y).astype(str)
print('x, y shape',x.shape, y.shape, cat_list.shape)      

In [ ]:
# group x
x_group = []
for i in range(0,len(folder_list)):
  x_group.append(x[i*no_of_files_in_each_class:(i+1)*no_of_files_in_each_class])
x_group = np.asarray(x_group)

#group y
y_group = np.unique(y)

In [ ]:
import numpy as np

train_size = int(len(folder_list) *0.8)
test_size = len(folder_list) - train_size
print(train_size, 'classes for training and', test_size, ' classes for testing')

# Training Split
x_train = x_group[:train_size]
y_train = y[:train_size*no_of_files_in_each_class]
cat_train = cat_list[:train_size*no_of_files_in_each_class]

x_train_full = x[:train_size*no_of_files_in_each_class]
x_test_full = x[train_size*no_of_files_in_each_class:]

# Validation Split
x_test = x_group[train_size:]
y_test = y[train_size*no_of_files_in_each_class:]
cat_test = cat_list[train_size*no_of_files_in_each_class:]

print('X&Y shape of training data :',x_train.shape, 'and', y_train.shape, cat_train.shape)
print('X&Y shape of testing data :' , x_test.shape, 'and', y_test.shape, cat_test.shape)

### Batch Generator
Here the idea is to make usuable batches for training the network. We need to create parallel inputs for the $A$ and $B$ images where the output is the distance. Here we make the naive assumption that if images are in the same group the similarity is 1 otherwise it is 0.

If we randomly selected all of the images we would likely end up with most images in different groups.

In [ ]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    out_img_a, out_img_b, out_score = [], [], []
    all_groups = list(range(len(in_groups)))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        out_img_a += [in_groups[c_idx][np.random.choice(range(in_groups[c_idx].shape[0]))] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            out_score += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            out_score += [0]*batch_halfsize
            
        out_img_b += [in_groups[c_idx][np.random.choice(range(in_groups[c_idx].shape[0]))] for c_idx in b_group_idx]
            
    return np.stack(out_img_a,0), np.stack(out_img_b,0), np.stack(out_score,0)

## Validate Data
Here we make sure the generator is doing something sensible, we show the images and their similarity percentage.

In [ ]:
pv_a, pv_b, pv_sim = gen_random_batch(x_train, 3)
fig, m_axs = plt.subplots(2, pv_a.shape[0], figsize = (12, 6))
for c_a, c_b, c_d, (ax1, ax2) in zip(pv_a, pv_b, pv_sim, m_axs.T):
    ax1.imshow(c_a)
    ax1.set_title('Image A')
    ax1.axis('off')
    ax2.imshow(c_b)
    ax2.set_title('Image B\n Similarity: %3.0f%%' % (100*c_d))
    ax2.axis('off')

# Feature Generation
Here we make the feature generation network to process images into features. The network starts off randomly initialized and will be trained to generate useful vector features from input images (_hopefully_)

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPool2D, Activation, Flatten, Dense, Dropout
img_in = Input(shape = x_train.shape[2:], name = 'FeatureNet_ImageInput')
n_layer = img_in
for i in range(2):
    n_layer = Conv2D(8*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = Conv2D(16*2**i, kernel_size = (3,3), activation = 'linear')(n_layer)
    n_layer = BatchNormalization()(n_layer)
    n_layer = Activation('relu')(n_layer)
    n_layer = MaxPool2D((2,2))(n_layer)
n_layer = Flatten()(n_layer)
n_layer = Dense(32, activation = 'linear')(n_layer)
n_layer = Dropout(0.5)(n_layer)
n_layer = BatchNormalization()(n_layer)
n_layer = Activation('relu')(n_layer)
feature_model = Model(inputs = [img_in], outputs = [n_layer], name = 'FeatureGenerationModel')
feature_model.summary()

# Siamese Model
We apply the feature generating model to both images and then combine them together to predict if they are similar or not. The model is designed to very simple. The ultimate idea is when a new image is taken that a feature vector can be calculated for it using the _FeatureGenerationModel_. All existing images have been pre-calculated and stored in a database of feature vectors. The model can be applied using a few vector additions and multiplications to determine the most similar images. These operations can be implemented as a stored procedure or similar task inside the database itself since they do not require an entire deep learning framework to run.

In [ ]:
from keras.layers import concatenate
img_a_in = Input(shape = x_train.shape[2:], name = 'ImageA_Input')
img_b_in = Input(shape = x_train.shape[2:], name = 'ImageB_Input')
img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)
combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(16, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(4, activation = 'linear')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Activation('relu')(combined_features)
combined_features = Dense(1, activation = 'sigmoid')(combined_features)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

In [ ]:
# setup the optimization process
similarity_model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['mae'])

## Before train
Here we visualize what the model does by taking a small sample of randomly selected A and B images the first half from the same category and the second from different categories. We then show the actual distance (0 for the same category and 1 for different categories) as well as the model predicted distance. The first run here is with a completely untrained network so we do not expect meaningful results.

In [ ]:
def show_model_output(nb_examples = 3):
    pv_a, pv_b, pv_sim = gen_random_batch(x_test, nb_examples)
    pred_sim = similarity_model.predict([pv_a, pv_b])
    fig, m_axs = plt.subplots(2, pv_a.shape[0], figsize = (12, 6))
    for c_a, c_b, c_d, p_d, (ax1, ax2) in zip(pv_a, pv_b, pv_sim, pred_sim, m_axs.T):
        ax1.imshow(c_a[:,:,:])
        ax1.set_title('Image A\n Actual: %3.0f%%' % (100*c_d))
        ax1.axis('off')
        ax2.imshow(c_b[:,:,:])
        ax2.set_title('Image B\n Predicted: %3.0f%%' % (100*p_d))
        ax2.axis('off')
    return fig
# a completely untrained model
_ = show_model_output()

In [ ]:
# make a generator out of the data
def siam_gen(in_groups, batch_size = 64):
    while True:
        pv_a, pv_b, pv_sim = gen_random_batch(x_train, batch_size//2)
        yield [pv_a, pv_b], pv_sim
        
# we want a constant validation group to have a frame of reference for model performance
valid_a, valid_b, valid_sim = gen_random_batch(x_test, 1024)
loss_history = similarity_model.fit_generator(siam_gen(x_train), 
                               steps_per_epoch = 500,
                               validation_data=([valid_a, valid_b], valid_sim),
                                              epochs = 50,
                                             verbose = True)

similarity_model.save('similarity_model_latest.h5')

###After train

In [ ]:
_ = show_model_output()

#Prediction


In [ ]:
test_df = pd.read_csv("/content/test-sample.csv")
test_df

In [ ]:
def read_im(i):
  # Display image 
  URL = test_df.url[i]

  with urllib.request.urlopen(URL) as url:
    f = io.BytesIO(url.read())
  img = Image.open(f)

  # Dimension
  w, h = np.array(img).shape[1], np.array(img).shape[0]
  shape = [(int(test_df.tl_x[i]*w), int(test_df.tl_y[i]*h)), (int(test_df.br_x[i]*w), int(test_df.br_y[i]*h))] 

  ip = img.crop((int(test_df.tl_x[i]*w), int(test_df.tl_y[i]*h), int(test_df.br_x[i]*w), int(test_df.br_y[i]*h)))
  # display(ip)
  ip = ip.convert('RGB').resize((100, 100))
  ip = np.asarray(ip)/255.0
  return ip

In [ ]:
#test
ip = read_im(0)
plt.imshow(ip)

In [ ]:
# similarity_model.load_weights('/content/weight/weights.3000.h5')

###x_group  and y_group

In [ ]:
x_test_submit = []
submit = []
x_test_submit.append(np.zeros((24, 100, 100, 3)))
x_test_submit.append(np.zeros((24, 100, 100, 3)))
for i in range(len(test_df)):
  ip = read_im(i)
  for j in range(24):
    x_test_submit[0][j] = ip
    index = random.randint(0,no_of_files_in_each_class-1)
    x_test_submit[1][j] = x_group[j][index]

  pred = similarity_model.predict(x_test_submit)
  submit.append(y_group[np.argmax(pred)])
  # print(pred)
  # print(y_group[np.argmax(pred)])

In [ ]:
submit_df = pd.DataFrame()
submit_df['key']=submit
submit_df.to_csv('machima_3.csv')

#Examining the Features
Here we aim to answer the more general question: did we generate useful features with the Feature Generation model? And how can we visualize this.

In [ ]:
x_test_features = feature_model.predict(x_test_full, verbose = True, batch_size=128)

## Neighbor Visualization
For this we use the TSNE neighborhood embedding to visualize the features on a 2D plane and see if it roughly corresponds to the groups. We use the test data for this example as well since the training has been contaminated

In [ ]:
%%time
from sklearn.manifold import TSNE
tsne_obj = TSNE(n_components=2,
                         init='pca',
                         random_state=101,
                         method='barnes_hut',
                         n_iter=500,
                         verbose=2)
tsne_features = tsne_obj.fit_transform(x_test_features)

In [ ]:
obj_categories = y_group
colors = plt.cm.rainbow(np.linspace(0, 1, 24))
plt.figure(figsize=(10, 10))

for c_group, (c_color, c_label) in enumerate(zip(colors, obj_categories)):
    plt.scatter(tsne_features[np.where(y_test == y_group[c_group]), 0],
                tsne_features[np.where(y_test == y_group[c_group]), 1],
                marker='o',
                color=c_color,
                linewidth='1',
                alpha=0.8,
                label=c_label)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('t-SNE on Testing Samples')
plt.legend(loc='best')
plt.savefig('clothes-dist.png')
plt.show(block=False)

In [ ]:
similarity_model.save('similarity_model_latest.h5')